In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import cv2 as cv
from PIL import Image

In [2]:
# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')


In [3]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0
...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0


In [4]:
import os

# Assuming you have loaded your schedule data into a DataFrame called 'schedule_df'
# and you have a dictionary mapping names to image filenames called 'name_to_image'

# Example schedule data (replace this with your actual data)
schedule_data = {
    'Name': ['شاهين ميرو',
 'شاهين ميرو',
 'أيمن سمارة',
 'أيمن سمارة',
 'رندة ناصر',
 'رندة ناصر',
 'مصطفى ملا علي',
 'مصطفى ملا علي',
 'نهلة أبو قورة',
 'نهلة أبو قورة',
 'لطيفة إبراهيم',
 'لطيفة إبراهيم',
 'فاطمة فرحان',
 'فاطمة فرحان',
 'صبحي الوزان',
 'صبحي الوزان',
 'هبة لولو',
 'هبة لولو',
 'خديجة كبة',
 'خديجة كبة',
 'عبدالرزاق بويضاني',
 'عبدالرزاق بويضاني',
 'أحمد طحان',
 'أحمد طحان',
 'حليمة شيخو',
 'حليمة شيخو',
 'علي الحسين',
 'علي الحسين',
 'محي الدين خاشوق',
 'محي الدين خاشوق',
 'دلال محمود',
 'دلال محمود',
 'نضال جريج',
 'نضال جريج',
 'فاطمة الخلف',
 'فاطمة الخلف',
 'نجاح النهار',
 'نجاح النهار',
 'مروة محجوب',
 'مروة محجوب',
 'زهير الملاح',
 'زهير الملاح',
 'باسل الباش',
 'باسل الباش',
 'نزار الحلو ',
 'نزار الحلو ',
 'ولاء حماد',
 'ولاء حماد',
 'حسين الحلقي',
 'حسين الحلقي',
 'حصة الحمد',
 'حصة الحمد',
 'ايمان الزعبي',
 'ايمان الزعبي',
 'سلفانة أبو عسلي',
 'سلفانة أبو عسلي',
 'وسام كاشي',
 'وسام كاشي',
 'صالح الهزاع',
 'صالح الهزاع',
 'راما نصرالله',
 'راما نصرالله',
 'أسرار صنوبر',
 'أسرار صنوبر',
 'حنان الابراهيم',
 'حنان الابراهيم',
 'عبدالله الغزولي ',
 'عبدالله الغزولي ',
 'نور عبدالله ',
 'نور عبدالله ',
 'جميلة عبيد',
 'جميلة عبيد',
 'عبدالرحيم حماد',
 'عبدالرحيم حماد',
 'جانيت ضاهر',
 'جانيت ضاهر',
 'فاطمة عوض',
 'فاطمة عوض',
 'حسن القاموع',
 'حسن القاموع',
 'محمد هلال',
 'محمد هلال',
 'عنود الحميد',
 'عنود الحميد',
 'زريفة المحمد',
 'زريفة المحمد',
 'هيثم قزاز',
 'هيثم قزاز',
 'حمود غشام',
 'حمود غشام'],'Eye':[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1]
    
}
schedule_df = pd.DataFrame(schedule_data)

# Example directory containing images
image_dir = "Data/"

# Example function to load image data
def load_image(filename):
    with open(os.path.join(image_dir, filename), "rb") as f:
        # return f.read()
        return f'{image_dir}{filename}'

# Example dictionary mapping names to image filenames
# You need to populate this dictionary with your actual filenames
name_to_image = {
    ('شاهين ميرو', 0): 'SHAHEN_MERO Optic nerve.png_ left.png',
    ('شاهين ميرو', 1): 'SHAHEN_MERO Optic nerve.png_ right.png',
    ('أيمن سمارة', 1): 'AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
    ('رندة ناصر', 0): 'RAMA_NASR_ALL Optic nerve.png_ left.png',
    ('رندة ناصر', 1): 'RANDA_NASER_2 Optic nerve.png_ right.png',
    ('مصطفى ملا علي', 0): 'MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
    ('مصطفى ملا علي', 1): 'MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
    ('نهلة أبو قورة', 0): '6361_NAHLA_ABO_QORA_left.png',
    ('لطيفة إبراهيم', 0): 'LATEFA_IBRAHEM Optic nerve.png_ left.png',
    ('لطيفة إبراهيم', 1): 'LATEFA_IBRAHEM Optic nerve.png_ right.png',
    ('فاطمة فرحان', 0): 'FATEMA_FARHAN Optic nerve.png_ left.png',
    ('فاطمة فرحان', 1): 'FATEMA_FARHAN Optic nerve.png_ right.png',
    ('صبحي الوزان', 0): 'SOBHI_WAZAN Optic nerve.png_ left.png',
    ('صبحي الوزان', 1): 'SOBHI_WAZAN Optic nerve.png_ right.png',
    ('هبة لولو', 0): 'HIBA_LOLO Optic nerve.png_ left.png',
    ('هبة لولو', 1): 'HIBA_LOLO Optic nerve.png_ right.png',
    ('خديجة كبة', 0): 'KHADEGA_KABEH Optic nerve.png_ left.png',
    ('خديجة كبة', 1): 'KHADEGA_KABEH Optic nerve.png_ right.png',
    ('عبدالرزاق بويضاني', 0): 'ABD_ALRAZAQ_BOEDANI Optic nerve.png_ left.png',
    ('عبدالرزاق بويضاني', 1): 'ABD_ALRAZAQ_BOEDANI Optic nerve.png_ right.png',
    ('أحمد طحان', 0): 'AHMAD_TAHAN Optic nerve.png_ left.png',
    ('أحمد طحان', 1): 'AHMAD_TAHAN Optic nerve.png_ right.png',
    ('حليمة شيخو', 0): '5919_HALEMA_SHEKHO Optic nerve.png_ left.png',
    ('حليمة شيخو', 1): '5919_HALEMA_SHEKHO Optic nerve.png_ right.png',
    ('علي الحسين', 0): 'ALI_HOSEN Optic nerve.png_ left.png',
    ('علي الحسين', 1): 'ALI_HOSEN Optic nerve.png_ right.png',
    ('محي الدين خاشوق', 0): 'MOHY_ALDEN_KHASHOQ Optic nerve.png_ left.png',
    ('محي الدين خاشوق', 1): 'MOHY_ALDEN_KHASHOQ Optic nerve.png_ right.png',
    ('دلال محمود', 0): 'DALAL_MAHMODOptic nerve.png_ left.png',
    ('دلال محمود', 1): 'DALAL_MAHMODOptic nerve.png_ right.png',
    ('نضال جريج', 0): 'NEDAL_JREJ Optic nerve.png_ left.png',
    ('نضال جريج', 1): 'NEDAL_JREJ Optic nerve.png_ right.png',
    ('فاطمة الخلف', 0): 'FATEMAH_KHALAF Optic nerve.png_ left.png',
    ('فاطمة الخلف', 1): 'FATEMAH_KHALAF Optic nerve.png_ right.png',
    ('نجاح النهار', 0): 'NAJAH_NAHAR  Optic nerve.png_ left.png',
    ('نجاح النهار', 1): 'NAJAH_NAHAR  Optic nerve.png_ right.png',
    ('مروة محجوب', 0): '5941_MARWA_MAHJOB Optic nerve.png_ left.png',
    ('مروة محجوب', 1): '5941_MARWA_MAHJOB Optic nerve.png_ right.png',
    ('زهير الملاح', 0): 'ZOHER_MALAH Optic nerve.png_ left.png',
    ('زهير الملاح', 1): 'ZOHER_MALAH Optic nerve.png_ right.png',
    ('باسل الباش', 0): 'BASEL_BASH Optic nerve.png_ left.png',
    ('باسل الباش', 1): 'BASEL_BASH Optic nerve.png_ right.png',
    ('نزار الحلو ', 0): 'NEZAR_HELOU Optic nerve.png_ left.png',
    ('نزار الحلو ', 1): 'NEZAR_HELOU Optic nerve.png_ right.png',
    ('ولاء حماد', 0): 'WALAA_HAMAD Optic nerve.png_ left.png',
    ('ولاء حماد', 1): 'WALAA_HAMAD Optic nerve.png_ right.png',
    ('حسين الحلقي', 0): 'HOSEN_HALAQI Optic nerve.png_ left.png',
    ('حسين الحلقي', 1): 'HOSEN_HALAQI Optic nerve.png_ right.png',
    ('حصة الحمد', 1): 'HASA_HAMAD_20240125_100552_Disc_3D_R_SINGLE_6mm_512x112_right.png',
    ('سلفانة أبو عسلي', 0): 'SELVANA_ABO_AS Optic nerve.png_ left.png',
    ('سلفانة أبو عسلي', 1): 'SELVANA_ABO_AS Optic nerve.png_ right.png',
    ('وسام كاشي', 0): 'WESAM_KASHI Optic nerve.png_ left.png',
    ('وسام كاشي', 1): 'WESAM_KASHI Optic nerve.png_ right.png',
    ('صالح الهزاع', 0): 'SALEH_HAZAA_20 Optic nerve.png_ left.png',
    ('صالح الهزاع', 1): 'SALEH_HAZAA_20 Optic nerve.png_ right.png',
    ('راما نصرالله', 0): 'RAMA_NASR_ALL Optic nerve.png_ left.png',
    ('راما نصرالله', 1): 'RAMA_NASR_ALL Optic nerve.png_ right.png',
    ('حنان الابراهيم', 0): 'HANAN_IBRAHEM Optic nerve.png_ left.png',
    ('حنان الابراهيم', 1): 'HANAN_IBRAHEM Optic nerve.png_ right.png',
    ('عبدالله الغزولي ', 0): 'ABD_ALLAH_GHAZOLI_20231007_11 Optic nerve.png_ left.png',
    ('عبدالله الغزولي ', 1): 'ABD_ALLAH_GHAZOLI_20231007_11 Optic nerve.png_ right.png',
    ('نور عبدالله ', 0): 'NOUR_ABD_ALLAH  Optic nerve.png_ left.png',
    ('نور عبدالله ', 1): 'NOUR_ABD_ALLAH  Optic nerve.png_ right.png',
    ('جميلة عبيد', 0): 'JAMELEH_ABED Optic nerve.png_ left.png',
    ('جميلة عبيد', 1): 'JAMELEH_ABED Optic nerve.png_ right.png',
    ('عبدالرحيم حماد', 0): 'ABD_ALRAHEM_HAMAD Optic nerve.png_ left.png',
    ('عبدالرحيم حماد', 1): 'ABD_ALRAHEM_HAMAD Optic nerve.png_ right.png',
    ('جانيت ضاهر', 0): 'JANET_DAHER Optic nerve.png_ left.png',
    ('جانيت ضاهر', 1): 'JANET_DAHER Optic nerve.png_ right.png',
    ('فاطمة عوض', 0): 'FATEMA_AWAD_20 Optic nerve.png_ left.png',
    ('فاطمة عوض', 1): 'FATEMA_AWAD_20 Optic nerve.png_ right.png',
    ('حسن القاموع', 0): 'HASAN_QAMOAAOptic nerve.png_ left.png',
    ('حسن القاموع', 1): 'HASAN_QAMOAAOptic nerve.png_ right.png',
    ('محمد هلال', 0): 'MOUHAMAD_HELAL_ Optic nerve.png_ left.png',
    ('محمد هلال', 1): 'MOUHAMAD_HELAL_ Optic nerve.png_ left.png',
    ('زريفة المحمد', 0): 'ZAREFA_MOUHAMAD_20231227_103707_Disc_3D_L_SINGLE_6mm_512x112_left.png',
    ('هيثم قزاز', 0): 'HAETHAM_QAZAZ Optic nerve.png_ left.png',
    ('هيثم قزاز', 1): 'HAETHAM_QAZAZ Optic nerve.png_ right.png',
}

# Function to get the image data based on the name
def get_image_data(name):
    filename = name_to_image.get(name ,None)
    if filename:
        return load_image(filename)
    else:
        return None

# Add a new column 'Image' to the DataFrame and fill it with image data
split_df['Image'] = schedule_df.apply(lambda row: get_image_data((row['Name'], row['Eye'])), axis=1)

# Display the DataFrame with image data
split_df.head(17)


,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Image
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,None
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,Data/RAMA_NASR_ALL Optic nerve.png_ left.png
5,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,1,1.0,0.4,0,14.0,Data/RANDA_NASER_2 Optic nerve.png_ right.png
6,زرق مزدوج أشد بالأيمن,1.0,60,0,مصطفى ملا علي,4.0,0,0.2,0.7,1,10.0,Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png
7,زرق مزدوج أشد بالأيمن,1.0,60,0,مصطفى ملا علي,4.0,1,0.1,0.7,1,14.0,Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png
8,زرق بالعينتين أشد بالأيمن,1.0,58,1,نهلة أبو قورة,5.0,0,0.5,NaN,1,8.0,Data/6361_NAHLA_ABO_QORA_left.png
9,زرق بالعينتين أشد بالأيمن,1.0,58,1,نهلة أبو قورة,5.0,1,1.0,NaN,1,11.0,None


In [5]:
split_df = split_df.drop(['Table 1', 'Gender', 'id', 'Age', 'Spectacles'], axis=1)

In [6]:
split_df = split_df.drop(['Name','Eye'], axis=1)

In [7]:
split_df.dropna(inplace=True)

In [8]:
split_df

,VA,CDR,Diagnosis,IOP,Image
0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,9.0,0.8,1,40.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
4,1.0,0.5,0,15.0,Data/RAMA_NASR_ALL Optic nerve.png_ left.png
5,1.0,0.4,0,14.0,Data/RANDA_NASER_2 Optic nerve.png_ right.png
...,...,...,...,...,...
80,0.7,0.8,0,13.0,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
81,9.0,0.8,1,16.0,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
84,0.2,0.8,1,10.0,Data/ZAREFA_MOUHAMAD_20231227_103707_Disc_3D_L...
86,0.3,0.5,1,15.0,Data/HAETHAM_QAZAZ Optic nerve.png_ left.png


In [9]:
len(split_df['Image'].index)

64

In [10]:
split_df['Image'][0]

'Data/SHAHEN_MERO Optic nerve.png_ left.png'

In [11]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Preprocess image function
# def preprocess_image(image_path, target_size=(224, 224)):
#     # Read the image
#     image = cv2.imread(image_path)
    
#     # Resize the image
#     image = cv2.resize(image, target_size)
    
#     # Convert the image to float32
#     image = image.astype(np.float32)
    
#     # Normalize pixel values to [0, 1]
#     image /= 255.0
    
#     return image

# Concatenation model
def create_concat_model(image_shape, num_numerical_features, num_classes):
    # Define input layers for images and numerical features
    image_input = Input(shape=image_shape)
    numerical_input = Input(shape=(num_numerical_features,))
    
    # Image processing layers
    conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    flatten_img = Flatten()(pool2)
    
    # Dense layers for numerical features
    dense1 = Dense(64, activation='relu')(numerical_input)
    
    # Concatenate image and feature layers
    concatenated = Concatenate()([flatten_img, dense1])
    
    # Additional layers for further processing or output
    dense2 = Dense(128, activation='relu')(concatenated)
    output = Dense(num_classes, activation='softmax')(dense2)
    
    # Define the model
    model = tf.keras.Model(inputs=[image_input, numerical_input], outputs=output)
    
    return model


# Data augmentation
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


augmented_images = []

for index,row in split_df.iterrows():
    # Load an image and preprocess it
    path = row['Image']
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (224, 224))  # Resize the image to the desired size
    image = image.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
    augmented_images.append(image)


    # # Apply data augmentation
    # for _ in range(5):  # Generate 5 augmented images
    #     augmented_image = data_generator.random_transform(image)
    #     augmented_images.append(augmented_image)





In [12]:
numerical_features = split_df.drop(['Diagnosis','Image'],axis=1)
numerical_features = np.array(numerical_features)
labels = np.array(split_df['Diagnosis'])  # NumPy array of shape (num_samples,)


In [13]:
len(numerical_features)

64

In [14]:
# Example usage:
image_shape = augmented_images[0].shape 
num_numerical_features = 3 
num_classes = 2  
concat_model = create_concat_model(image_shape, num_numerical_features, num_classes)

# Compile the model
concat_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
concat_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 109, 109, 64)         18496     ['max_pooling2d[0][0]']   

In [16]:
# Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [17]:
X_image_train, X_image_test, X_num_features_train, X_num_features_test, y_labels_train, y_labels_test = train_test_split(augmented_images, numerical_features, labels, test_size=0.2, random_state=42)

In [18]:
print(len(X_image_train))
print(len(X_image_test))
print(len(X_num_features_train))
print(len(X_num_features_test))
print(len(y_labels_train))
print(len(y_labels_test))

51
13
51
13
51
13


In [24]:
X_image_train, X_image_test, X_num_features_train, X_num_features_test, y_labels_train, y_labels_test = \
    train_test_split(augmented_images, numerical_features, labels, test_size=0.2, random_state=42)


print(len(X_image_train))
print(len(X_image_test))
print(len(X_num_features_train))
print(len(X_num_features_test))
print(len(y_labels_train))
print(len(y_labels_test))


51
13
51
13
51
13


In [25]:
y_labels_train.size

51

In [28]:
concat_model.fit(x=[X_image_train, X_num_features_train], y=y_labels_train, validation_data=([X_image_test, X_num_features_test], y_labels_test),
                  batch_size=32, epochs=10, )

ValueError: Data cardinality is ambiguous:
  x sizes: 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 51
  y sizes: 51
Make sure all arrays contain the same number of samples.